<a href="https://www.kaggle.com/code/scr0ll0/modeling-submission?scriptVersionId=154557742" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Imports/Downloads

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression, SGDClassifier

# Loading Data

Credit to Darek for the daigt-v2-train-dataset: https://www.kaggle.com/datasets/thedrcat/daigt-v2-train-dataset

In [2]:
train = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sample = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')

# Cleaning Data

In [3]:
#Filtering
train = train[['text', 'label']].copy()
sample['id'] = test['id']
test = test.drop(columns=['id', 'prompt_id'])

dev_x = train.drop(columns='label')
dev_y = train['label']

In [4]:
#Sampling
dev_x, dev_y = RandomUnderSampler(random_state=0).fit_resample(dev_x, dev_y)

In [5]:
#Cleaning
def clean_text(text):
    text = text.strip()
    return text
dev_x['text'] = dev_x['text'].apply(lambda x: clean_text(x))
test['text'] = test['text'].apply(lambda x: clean_text(x))

Credit to both pamilove_dl (https://www.kaggle.com/competitions/llm-detect-ai-generated-text/discussion/455701) and Damien Mourot (https://www.kaggle.com/competitions/llm-detect-ai-generated-text/discussion/455969) for suggesting to fit the TF-IDF on the test set

In [6]:
#Vectorizing
vector = TfidfVectorizer(stop_words='english')
#vector.fit(dev_x['text'])
vector.fit(test['text'])
dev_x = pd.DataFrame(vector.transform(dev_x['text']).toarray())
test = pd.DataFrame(vector.transform(test['text']).toarray())

In [7]:
#Scaling
scaler = StandardScaler()
scaler.fit(dev_x)
dev_x = scaler.transform(dev_x)
test = scaler.transform(test)

# Modeling

In [8]:
#Logistic Regression
model = LogisticRegression()
model.fit(dev_x, dev_y)

LogisticRegression()

In [9]:
#SGD Classifier

#model = SGDClassifier(loss='modified_huber', random_state=0)
#model.fit(dev_x, dev_y)

# Predictions

In [10]:
sample['generated'] = model.predict_proba(test)[:, 1]

In [11]:
sample.to_csv('submission.csv', index=False)